In [1]:
#Import Required Packages
import pandas as pd
import requests
import googlemaps
import numpy as np
from time import sleep

In [2]:
# Read in USGS Flood Data and establish ZIP column
floods = pd.read_csv('Hurricane.csv')

floods.drop(floods[floods['height_above_gnd'] > 30].index, inplace = True)

floods['zip'] = np.zeros(floods.shape[0])

In [3]:
# My Google Maps API key - deleted
mykey = ''

In [4]:
#Use Google Maps API to get ZIP code for every latitude longitude pair in flood dataset
gmaps = googlemaps.Client(key=mykey)

for i in floods.index[0:1]:#I truncated the index to show the cell runs without re-running entire query
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={floods.loc[i, 'latitude']},{floods.loc[i, 'longitude']}&result_type=postal_code&key={mykey}"
    print(f'Getting coordinates for index {i}')
    res = requests.get(url)
    a = res.json()['results']
    if a != []:
        floods.loc[i, 'zip'] = a[0]['address_components'][0]['long_name']
    sleep(1)
#floods.to_csv('HurricanewithZIP.csv')

Getting coordinates for index 0


In [5]:
#Aggregate average flood height per zip per event
zip_heights = floods.groupby(['zip', 'eventName']).mean()[['height_above_gnd']].reset_index()
zip_heights.drop(zip_heights[zip_heights['zip'] == 0].index, inplace = True)
zip_heights.to_csv('FloodHeightsZIP.csv')